##### nyc data from datatalk
https://github.com/DataTalksClub/nyc-tlc-data/?tab=readme-ov-file

In [16]:
import pyspark
from pyspark.sql import SparkSession
import pandas as pd
from pyspark.sql import types
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/11/30 23:44:20 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
!wc -l fhvhv_tripdata_2021-01.csv 

11908469 fhvhv_tripdata_2021-01.csv


In [ ]:
df= spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [ ]:
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [4]:
!wc -l head.csv

1001 head.csv


In [5]:
!head -n 10 head.csv

hvfhs_license_num,dispatching_base_num,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,SR_Flag
HV0003,B02682,2021-01-01 00:33:44,2021-01-01 00:49:07,230,166,
HV0003,B02682,2021-01-01 00:55:19,2021-01-01 01:18:21,152,167,
HV0003,B02764,2021-01-01 00:23:56,2021-01-01 00:38:05,233,142,
HV0003,B02764,2021-01-01 00:42:51,2021-01-01 00:45:50,142,143,
HV0003,B02764,2021-01-01 00:48:14,2021-01-01 01:08:42,143,78,
HV0005,B02510,2021-01-01 00:06:59,2021-01-01 00:43:01,88,42,
HV0005,B02510,2021-01-01 00:50:00,2021-01-01 01:04:57,42,151,
HV0003,B02764,2021-01-01 00:14:30,2021-01-01 00:50:27,71,226,
HV0003,B02875,2021-01-01 00:22:54,2021-01-01 00:30:20,112,255,


In [6]:
df_pandas = pd.read_csv("head.csv")

In [7]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [8]:
spark.createDataFrame(df_pandas).schema

/home/MERHAWI/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/home/MERHAWI/spark/spark-3.3.2-bin-hadoop3/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('pickup_datetime', StringType(), True), StructField('dropoff_datetime', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('SR_Flag', DoubleType(), True)])

In [9]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True), 
    types.StructField('pickup_datetime', types.TimestampType(), True), 
    types.StructField('dropoff_datetime', types.TimestampType(), True), 
    types.StructField('PULocationID', types.IntegerType(), True), 
    types.StructField('DOLocationID', types.IntegerType(), True), 
    types.StructField('SR_Flag', types.StringType(), True)]
)

In [10]:
df= spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [ ]:
df.show()

In [ ]:
df = df.repartition(24)

In [ ]:
df.write.parquet('fhvhv/2021/01')

In [11]:
df = spark.read.parquet('fhvhv/2021/01')

In [12]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



In [15]:
df.select('pickup_datetime','dropoff_datetime','PULocationID','DOLocationID') \
    .filter(df.hvfhs_license_num=='HV0003') \
    .show()

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-01-04 09:54:16|2021-01-04 10:20:02|         152|         193|
|2021-01-03 16:45:51|2021-01-03 16:58:38|         208|         213|
|2021-01-04 15:51:39|2021-01-04 16:03:16|         145|         202|
|2021-01-02 08:15:37|2021-01-02 08:19:39|          78|         169|
|2021-01-01 17:46:05|2021-01-01 18:10:31|         181|         107|
|2021-01-02 19:36:41|2021-01-02 19:43:00|         188|          72|
|2021-01-01 16:05:01|2021-01-01 16:26:27|          61|           4|
|2021-01-03 00:05:40|2021-01-03 00:13:45|         257|          54|
|2021-01-04 16:37:58|2021-01-04 16:46:13|          75|          75|
|2021-01-01 23:42:33|2021-01-02 00:06:29|         147|         175|
|2021-01-05 08:31:45|2021-01-05 08:38:00|          81|          81|
|2021-01-01 18:57:24|2021-01-01 19:01:41|       

In [28]:
def user_df_pyspark_function(basenum):
    """
    example user defined pyspark function
    """
    num = int(basenum[1:])
    if num % 7==0:
        return f's/{num:03x}'
    elif num % 3 ==0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [25]:
user_df_pyspark_function('B02682')

'a/a7a'

In [26]:
user_defined_pysp_fun = F.udf(user_df_pyspark_function, returnType=types.StringType())

In [27]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', user_defined_pysp_fun(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date','dropoff_date','PULocationID','DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/b3b| 2021-01-04|  2021-01-04|         152|         193|
|  e/b14| 2021-01-03|  2021-01-03|         208|         213|
|  e/b30| 2021-01-04|  2021-01-04|         145|         202|
|  e/95b| 2021-01-02|  2021-01-02|          78|         169|
|  e/9ce| 2021-01-02|  2021-01-02|          17|          73|
|  e/a39| 2021-01-01|  2021-01-01|         181|         107|
|  e/b38| 2021-01-02|  2021-01-02|         188|          72|
|  s/b13| 2021-01-01|  2021-01-01|          61|           4|
|  a/b49| 2021-01-03|  2021-01-03|         257|          54|
|  e/b47| 2021-01-04|  2021-01-04|          75|          75|
|  e/b35| 2021-01-01|  2021-01-02|         147|         175|
|  e/b32| 2021-01-05|  2021-01-05|          81|          81|
|  e/b42| 2021-01-01|  2021-01-01|          60|          60|
|  e/b48| 2021-01-04|  2